In [1]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [3]:
eia_api_key = "qjAQkgQvyQKzlWw3cGAL5UoBAv20MKF4QhIHbuXK"

In [4]:
print(f"Cheia mea API este: {eia_api_key}")


Cheia mea API este: qjAQkgQvyQKzlWw3cGAL5UoBAv20MKF4QhIHbuXK


In [42]:

API = eia_api_key
URL = "https://api.eia.gov/v2/international/data/"

def get_eia_product(pid):
    rows = []
    offset = 0

    while True:
        params = {
            "api_key": API,
            "frequency": "annual",
            "data[0]": "value",
            "start": "2000",
            "end": "2025",
            "facets[productId][]": pid,
            "sort[0][column]": "period",
            "sort[0][direction]": "desc",
            "offset": offset,
            "length": 5000
        }

        res = requests.get(URL, params=params).json()
        data = res["response"]["data"]

        rows.extend(data)

        if len(data) < 5000:
            break

        offset += 5000

    return pd.DataFrame(rows)



In [43]:
df_list = []
PRODUCT_IDS = [2, 30, 31, 32, 29, 37, 116, 33, 27, 44, 4008, 47, 4701, 4702] 

for pid in PRODUCT_IDS:
    df_list.append(get_eia_product(pid))

df = pd.concat(df_list, ignore_index=True)


In [44]:
print(df.shape)
df.head()

(332324, 14)


,period,productId,productName,activityId,activityName,countryRegionId,countryRegionName,countryRegionTypeId,countryRegionTypeName,dataFlagId,dataFlagDescription,unitName,value,unit
0,2023,2,Electricity,2,Consumption,ABW,Aruba,c,Country,None,None,billion kilowatthours,.824035908,BKWH
1,2023,2,Electricity,2,Consumption,AFG,Afghanistan,c,Country,None,None,billion kilowatthours,6.46823218,BKWH
2,2023,2,Electricity,2,Consumption,AGO,Angola,c,Country,None,None,billion kilowatthours,16.213833931,BKWH
3,2023,2,Electricity,2,Consumption,ALB,Albania,c,Country,None,None,billion kilowatthours,7.489578147,BKWH
4,2023,2,Electricity,2,Consumption,ARE,United Arab Emirates,c,Country,None,None,billion kilowatthours,157.973627617,BKWH


In [45]:
df["countryRegionName"].unique()


array(['Aruba', 'Afghanistan', 'Angola', 'Albania',
       'United Arab Emirates', 'Argentina', 'Armenia', 'American Samoa',
       'Antarctica', 'Antigua and Barbuda', 'Australia', 'Austria',
       'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso',
       'Bangladesh', 'Bulgaria', 'Bahrain', 'The Bahamas',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland', 'Chile',
       'China', "Cote d'Ivoire", 'Cameroon', 'Congo-Kinshasa',
       'Congo-Brazzaville', 'Cook Islands', 'Colombia', 'Comoros',
       'Cabo Verde', 'Costa Rica', 'Former Czechoslovakia', 'Cuba',
       'Cayman Islands', 'Cyprus', 'Czechia', 'Germany, East', 'Germany',
       'Germany, West', 'Djibouti', 'Dominica', 'Denmark',
       'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt', 'Eritrea',
       'Western Sahara', 'Spain', 'Estonia', 'Ethiopia', 'Finland',
    

In [46]:
df_africa_countries = df[
    (df["countryRegionTypeName"] == "Country") & 
    (df["countryRegionName"].str.contains("Africa"))
].copy()


In [47]:
df_africa_countries.head()

,period,productId,productName,activityId,activityName,countryRegionId,countryRegionName,countryRegionTypeId,countryRegionTypeName,dataFlagId,dataFlagDescription,unitName,value,unit
31,2023,2,Electricity,2,Consumption,CAF,Central African Republic,c,Country,None,None,billion kilowatthours,.132105202,BKWH
224,2023,2,Electricity,2,Consumption,ZAF,South Africa,c,Country,None,None,billion kilowatthours,194.977980159,BKWH
386,2023,2,Electricity,3,Imports,CAF,Central African Republic,c,Country,None,None,billion kilowatthours,0,BKWH
387,2023,2,Electricity,3,Imports,CAF,Central African Republic,c,Country,None,None,million metric tons of oil equivalent,0,MTOE
388,2023,2,Electricity,3,Imports,CAF,Central African Republic,c,Country,None,None,quadrillion Btu,0,QBTU


In [50]:
df_africa_countries.info()
df_africa_countries["productName"].value_counts()
df_africa_countries["activityName"].value_counts()
df_africa_countries["period"].unique()


<class 'pandas.core.frame.DataFrame'>
Index: 2530 entries, 31 to 332286
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   period                 2530 non-null   object
 1   productId              2530 non-null   object
 2   productName            2530 non-null   object
 3   activityId             2530 non-null   object
 4   activityName           2530 non-null   object
 5   countryRegionId        2530 non-null   object
 6   countryRegionName      2530 non-null   object
 7   countryRegionTypeId    2530 non-null   object
 8   countryRegionTypeName  2530 non-null   object
 9   dataFlagId             387 non-null    object
 10  dataFlagDescription    387 non-null    object
 11  unitName               2530 non-null   object
 12  value                  2530 non-null   object
 13  unit                   2530 non-null   object
dtypes: object(14)
memory usage: 296.5+ KB


array(['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016',
       '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008',
       '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2024'],
      dtype=object)

In [ ]:
df_africa_countries.to_csv("eia_africa_countries_raw.csv", index=False)


Saved eia_africa_countries_raw.csv
